**This notebook is copy of this one https://www.kaggle.com/steamedsheep/yolov5-is-all-you-need I just follow the advices from discussion and changed the input size from 3000 to 10000**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm import tqdm
import sys

sys.path.append('../input/tensorflow-great-barrier-reef')

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
model = torch.hub.load('../input/yolov5-lib-ds', 
                       'custom', 
                       path='../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt',
                       source='local',
                       force_reload=True)  # local repo
model.conf = 0.3

In [ ]:
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    anno = ''
    r = model(img, size=1000, augment=False)
    if r.pandas().xyxy[0].shape[0] == 0:
        anno = ''
    else:
        for idx, row in r.pandas().xyxy[0].iterrows():
            if row.confidence > 0.35:
                anno += '{} {} {} {} {} '.format(row.confidence, int(row.xmin), int(row.ymin), int(row.xmax-row.xmin), int(row.ymax-row.ymin))
#                 pred.append([row.confidence, row.xmin, row.ymin, row.xmax-row.xmin, row.ymax-row.ymin])
    pred_df['annotations'] = anno.strip(' ')
    env.predict(pred_df)